In [67]:
import os
import librosa
from librosa import display
import numpy as np
from matplotlib import pyplot as plt
import time
import random


import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import datetime
from matplotlib import pyplot as plt
from pretty_confusion_matrix import pp_matrix_from_data

# Listing all samples

In [68]:
DATASET_PATH = "samples"
TRAIN_VAL_SPLIT = 0.1

samples = os.listdir(DATASET_PATH)
random.shuffle(samples)

# EmotionsDict = {'N': 'Neutral', 'F':'Fear', 'H':'Happiness', 'S':'Sadness', 'W':'Surprise', 'A':'Angry'}
EmotionsDict = {'N': 0, 'F': 1, 'H': 2, 'S': 3, 'W': 4, 'A': 5}
emotions = [EmotionsDict[x[3]] for x in samples]

# Preprocessing data and creating features

In [69]:
sr = 44100

SIG_LENGTH = 200

mfccs = []

for s in samples:
    t1 = time.time()

    sig, sr = librosa.load(f'{DATASET_PATH}/{s}', sr=sr)
    sig, _ = librosa.effects.trim(sig) # trimming beginning and ending silence


    # MFCC config
    WinLen = int(0.040 * sr) # 40 milisecond
    WinHop = WinLen // 2
    
    sig_mfcc = librosa.feature.mfcc(y=librosa.power_to_db(sig), sr=sr, n_mfcc=12, fmax=sr//2)

    # cutting every longer sequence than SIG_LENGTH and padding any sequence shorter than SIG_LENGTH
    if sig_mfcc.shape[1] < SIG_LENGTH:
        sig_mfcc = np.pad(sig_mfcc, [(0, 0), (0, SIG_LENGTH - sig_mfcc.shape[1])], mode='symmetric')
    elif sig_mfcc.shape[1] > SIG_LENGTH:
        sig_mfcc = sig_mfcc[:, :SIG_LENGTH]
    
    sig_mfcc = sig_mfcc.transpose()
    mfccs.append(sig_mfcc)

In [70]:
mfccs = np.array(mfccs)
emotions = np.array(emotions)

ds_len = mfccs.shape[0]
split_point = int(ds_len*(1-TRAIN_VAL_SPLIT))

avg = mfccs.mean()
std = mfccs.std()

mfccs = (mfccs - avg) / std # normalization

# mfccs = np.expand_dims(mfccs, axis=-1)

train_x = mfccs[:split_point]
train_y = emotions[:split_point]

val_x = mfccs[split_point:]
val_y = emotions[split_point:]


# print(mfccs.mean(), mfccs.std())

In [71]:
BATCH_SIZE = 64
LAYERS_ACTIVATION = 'relu'
LOSS_FUNCTION = 'sparse_categorical_crossentropy'

EPOCHS = 200

def get_model():
    model = tf.keras.Sequential([
              tf.keras.layers.Conv1D(256, 11),
              tf.keras.layers.MaxPool1D(3),
              tf.keras.layers.Conv1D(128, 7),
              tf.keras.layers.MaxPool1D(3),
              tf.keras.layers.Conv1D(64, 7),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(64, activation=LAYERS_ACTIVATION),
              tf.keras.layers.Dense(6, activation=tf.keras.activations.softmax)               
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss=LOSS_FUNCTION, metrics=['accuracy'])
    return model

model = get_model()
history = model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/200
21/21 [==============================] - 4s 128ms/step - loss: 1.4651 - accuracy: 0.2746 - val_loss: 1.3977 - val_accuracy: 0.2617
Epoch 2/200
21/21 [==============================] - 3s 120ms/step - loss: 1.3864 - accuracy: 0.3082 - val_loss: 1.3889 - val_accuracy: 0.2349
Epoch 3/200
21/21 [==============================] - 2s 119ms/step - loss: 1.3751 - accuracy: 0.2910 - val_loss: 1.4289 - val_accuracy: 0.2215
Epoch 4/200
21/21 [==============================] - 3s 124ms/step - loss: 1.3619 - accuracy: 0.3269 - val_loss: 1.3814 - val_accuracy: 0.3087
Epoch 5/200
21/21 [==============================] - 3s 136ms/step - loss: 1.3544 - accuracy: 0.3291 - val_loss: 1.3833 - val_accuracy: 0.2953
Epoch 6/200
21/21 [==============================] - 2s 116ms/step - loss: 1.3471 - accuracy: 0.3358 - val_loss: 1.3639 - val_accuracy: 0.3020
Epoch 7/200
21/21 [==============================] - 2s 114ms/step - loss: 1.3387 - accuracy: 0.3537 - val_loss: 1.3485 - val_accuracy: 0.3087